# Analyzing borrowers’ risk of defaulting
#### Project description:
* The project purpose is report for a bank’s loan division to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan.
* The report will be considered when building the **credit score** of a potential customer. 
* The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

#### Table of contents: 

* [Open the data and study the general information:](#Open-the-data-and-study-the-general-information:)

* [Data exploration:](#Data-exploration:)

* [Data Preprocessing:](#Data-Preprocessing-:)

   * a.[Data transformation](#Data-transformation:)
   
   * b.[Working with missing values](#Working-with-missing-values)
   
   * c.[Restoring missing values in total_income:](#Restoring-missing-values-in-total_income:)
   
   * d.[Restoring values in days_employed:](#Restoring-values-in-days_employed:)
   
   * e.[Categorization-of-data:](#Categorization-of-data:)

* [Checking the Hypotheses:](#Checking-the-Hypotheses:)

* [General Conclusion](#General-Conclusion)


#### Open the data and study the general information:

In [1]:
# Loading all the libraries
import pandas as pd
import numpy as np

# Load the data
try:
    credit_scoring = pd.read_csv('credit_scoring_eng.csv')
except:
    credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv') 

#### Data exploration: 

##### Description of the data:
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan


In [2]:
# Let's see how many rows and columns our dataset has

credit_scoring.shape


(21525, 12)

In [3]:
# let's print the first N rows
credit_scoring.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


* There are negative values in days_employed column, and capital letters problem on the education column

In [4]:
# Get info on data

credit_scoring.info()
print('*********************************')
credit_scoring.describe(include='all').T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
*********************************


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
children,21525,NaN,NaN,NaN,0.538908,1.38159,-1,0,0,1,20
days_employed,19351,NaN,NaN,NaN,63046.5,140827,-18388.9,-2747.42,-1203.37,-291.096,401755
dob_years,21525,NaN,NaN,NaN,43.2934,12.5746,0,33,42,53,75
education,21525,15,secondary education,13750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education_id,21525,NaN,NaN,NaN,0.817236,0.548138,0,1,1,1,4
family_status,21525,5,married,12380,NaN,NaN,NaN,NaN,NaN,NaN,NaN
family_status_id,21525,NaN,NaN,NaN,0.972544,1.42032,0,0,0,1,4
gender,21525,3,F,14236,NaN,NaN,NaN,NaN,NaN,NaN,NaN
income_type,21525,8,employee,11119,NaN,NaN,NaN,NaN,NaN,NaN,NaN
debt,21525,NaN,NaN,NaN,0.0808827,0.272661,0,0,0,0,1


* There are missing values in 'days_employed' and 'total_income' columns, because there's less then 21525 full cells in both of them

##### Data Preprocessing : 

In [5]:
# Let's look at the filtered table with missing values in the the first column with missing data
credit_scoring[credit_scoring['days_employed'].isnull() == 1]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


* The missing value's seem to be symmetric, but we need to investigate more.

In [6]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.
print(len(credit_scoring[credit_scoring['days_employed'].isnull() & credit_scoring['total_income'].notnull()]))
print(len(credit_scoring[credit_scoring['days_employed'].notnull() & credit_scoring['total_income'].isnull()]))
print()
print(credit_scoring[credit_scoring['days_employed'].isnull() & credit_scoring['total_income'].isnull()].count())

0
0

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


**Intermediate conclusion :**

* The number of rows in the filtered table match the number of missing values, and from looking in the columns name we can conclude that "total_income" depend on "days employed"

* I looked in the data info for problems, there are missing value on two columns. Next Im going to look for connection between the missing values and the ther catagories

* If there is no connection, I'll fill the data with mean of the columns

* I'll look if ther's duplicates, typo's, and then I'll try to analyze the data

In [7]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
credit_scoring[credit_scoring['days_employed'].isnull() & credit_scoring['total_income'].isnull()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [8]:
# Checking distribution
missing_row = len(credit_scoring[credit_scoring['days_employed'].isnull() & credit_scoring['total_income'].isnull()])
length = len(credit_scoring['days_employed'])
print(missing_row/length)

0.10099883855981417


* Clients who didn't work didn't have any income, those client are 10.1% of all client

* Possible reasons for missing values is  malefunction.

In [9]:
# Checking the distribution in the whole dataset
days_employed = credit_scoring[credit_scoring['days_employed'].isnull()]
income_type_non_days_employed = days_employed.groupby('income_type')['dob_years'].count()
print("Nan values in days_employed in income type:")
print(income_type_non_days_employed)
print('******')
income_type_cat = credit_scoring.groupby('income_type')['days_employed'].count()
print("total income type categorized:")
print(income_type_cat)
print('******')
print("distribution of Nan values in days_employed in categorized income_type")
print(income_type_non_days_employed/income_type_cat)

Nan values in days_employed in income type:
income_type
business          508
civil servant     147
employee         1105
entrepreneur        1
retiree           413
Name: dob_years, dtype: int64
******
total income type categorized:
income_type
business                        4577
civil servant                   1312
employee                       10014
entrepreneur                       1
paternity / maternity leave        1
retiree                         3443
student                            1
unemployed                         2
Name: days_employed, dtype: int64
******
distribution of Nan values in days_employed in categorized income_type
income_type
business                       0.110990
civil servant                  0.112043
employee                       0.110346
entrepreneur                   1.000000
paternity / maternity leave         NaN
retiree                        0.119954
student                             NaN
unemployed                          NaN
dtype: float64

* in 4 out 8 income type the distribution here is similar to the general distribution. 
* I can calculate the mean and the median per income type. 


In [10]:
# Check for other reasons and patterns that could lead to missing value

days_employed = credit_scoring[credit_scoring['days_employed'].isnull()]
income_type_non_family_status = days_employed.groupby('family_status')['dob_years'].count()
print("Nan values in family_status:")
print(income_type_non_family_status)
print('******')
income_type_cat = credit_scoring.groupby('family_status')['days_employed'].count()
print("total income type categorized:")
print(income_type_cat)
print('******')
print("distribution of Nan values in categorized family_status")
print(income_type_non_family_status/income_type_cat)

Nan values in family_status:
family_status
civil partnership     442
divorced              112
married              1237
unmarried             288
widow / widower        95
Name: dob_years, dtype: int64
******
total income type categorized:
family_status
civil partnership     3735
divorced              1083
married              11143
unmarried             2525
widow / widower        865
Name: days_employed, dtype: int64
******
distribution of Nan values in categorized family_status
family_status
civil partnership    0.118340
divorced             0.103416
married              0.111011
unmarried            0.114059
widow / widower      0.109827
dtype: float64


In [11]:
# Checking for other patterns - education
days_employed = credit_scoring[credit_scoring['days_employed'].isnull()]
income_type_non_education = days_employed.groupby('education')['dob_years'].count()
print("Nan values in education:")
print(income_type_non_education)
print('******')
income_type_cat = credit_scoring.groupby('education')['days_employed'].count()
print("total income type categorized:")
print(income_type_cat)
print('******')
print("distribution of Nan values in categorized education")
print(income_type_non_education/income_type_cat)

Nan values in education:
education
BACHELOR'S DEGREE        23
Bachelor's Degree        25
PRIMARY EDUCATION         1
Primary Education         1
SECONDARY EDUCATION      67
SOME COLLEGE              7
Secondary Education      65
Some College              7
bachelor's degree       496
primary education        19
secondary education    1408
some college             55
Name: dob_years, dtype: int64
******
total income type categorized:
education
BACHELOR'S DEGREE        251
Bachelor's Degree        243
GRADUATE DEGREE            1
Graduate Degree            1
PRIMARY EDUCATION         16
Primary Education         14
SECONDARY EDUCATION      705
SOME COLLEGE              22
Secondary Education      646
Some College              40
bachelor's degree       4222
graduate degree            4
primary education        231
secondary education    12342
some college             613
Name: days_employed, dtype: int64
******
distribution of Nan values in categorized education
education
BACHELOR'S DE

**Conclusions**


I didn't found any patterns. the distribution is around 10 precents in some cases and 0 in others.

I tried to find correlation between total income to education, family status and income type, its make sense thet conection can be made, but apperntly it can't, at least in this case.

I need to address different types of issues: duplicates, different registers, incorrect artifacts, and missing values.

#### Data transformation:


In [12]:
# Let's see all values in education column to check if and what spellings will need to be fixed
print(credit_scoring['education'].unique())
print('*****')
print(credit_scoring['education_id'].unique())


["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']
*****
[0 1 2 3 4]


In [13]:
# Fix the registers if required
credit_scoring['education'] = credit_scoring['education'].str.lower()

In [14]:
# Checking all the values in the column to make sure we fixed them

print(credit_scoring['education'].unique())

["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


In [15]:
# Let's see the distribution of values in the `children` column
print(credit_scoring['children'].value_counts())
print('*********')
print('the problematic values precentage:')
print((47+76)/len(credit_scoring))

print(credit_scoring[credit_scoring['children']==-1])
print(credit_scoring[credit_scoring['children']==20])

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
*********
the problematic values precentage:
0.005714285714285714
       children  days_employed  dob_years            education  education_id  \
291          -1   -4417.703588         46  secondary education             1   
705          -1    -902.084528         50  secondary education             1   
742          -1   -3174.456205         57  secondary education             1   
800          -1  349987.852217         54  secondary education             1   
941          -1            NaN         57  secondary education             1   
1363         -1   -1195.264956         55  secondary education             1   
1929         -1   -1461.303336         38  secondary education             1   
2073         -1   -2539.761232         42  secondary education             1   
3814         -1   -3045.290443         26  secondary education             1   
4


0.05% of the column has problematic values in it, it's probably typo, for -1 and 20. I'll change -1 to 1, and 20 to 2.

In [16]:
# [fix the data based on your decision]

credit_scoring.loc[credit_scoring['children']==-1, 'children'] =1
credit_scoring.loc[credit_scoring['children']==20, 'children'] =2

In [17]:
# Checking the `children` column again to make sure it's all fixed

print(credit_scoring['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


`days_employed` column could have negative values or too high values

In [18]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
neg_days_employed = len(credit_scoring[credit_scoring['days_employed'] < 0])
print (neg_days_employed)
print(neg_days_employed/len(credit_scoring))

15906
0.7389547038327526


In [19]:
# Address the problematic values, if they exist

credit_scoring['days_employed']= abs(credit_scoring['days_employed'])


assuming that individual can work for 75 years tops lets check how meny rows represnt people who worked more then that- 328,500 days:

In [20]:
# Check the result - make sure it's fixed
tolong_days_employed = len(credit_scoring[credit_scoring['days_employed'] < 328500])
print(tolong_days_employed)

15906


Its to large portion to ignore, I don't think that we need to use this data anyway. In real word I would notify someone above me that something wrong with the data base.

 At the client's age the problem could be: negative values or zero's

In [21]:
# Check the `dob_years` for suspicious values and count the percentage

neg_dob_years= len(credit_scoring[credit_scoring['dob_years'] == 0])
print('number of dob_years equal zero:')
print(neg_dob_years)
print("precentage of zero's in dob_years:")
precentage = neg_dob_years/len(credit_scoring)
print(precentage)


number of dob_years equal zero:
101
precentage of zero's in dob_years:
0.004692218350754936


I replace the 0's with the median. the mean is 43.3 and the median is 42, those values are similar, so I chose the 42, because its round.

In [22]:
# Address the issues in the `dob_years` column, if they exist
dob_mean = credit_scoring['dob_years'].mean()
print("mean:",dob_mean)
dob_median = credit_scoring['dob_years'].median()
print("median:", dob_median)
credit_scoring.loc[credit_scoring['dob_years'] == 0,'dob_years'] = dob_median

mean: 43.29337979094077
median: 42.0


In [23]:
# Check the result - make sure it's fixed
print(len(credit_scoring[credit_scoring['dob_years'] == 0]))

0


In [24]:
# Let's see the values for the column
print(credit_scoring['family_status'].value_counts())


married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64


In [25]:
# Let's see the values in the column
print(credit_scoring['gender'].value_counts())
credit_scoring[credit_scoring['gender'] == 'XNA']

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24.0,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


I'll change the XNA into F because its more likely the the case.

In [26]:
# Address the problematic values, if they exist
credit_scoring.loc[credit_scoring['gender'] == 'XNA','gender']='F'

In [27]:
# Check the result - make sure it's fixed
credit_scoring[credit_scoring['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [28]:
# Let's see the values in the column
print(credit_scoring['income_type'].value_counts())
print('*********')
print(credit_scoring['income_type'].isnull==1)

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64
*********
False


In [29]:
# Checking duplicates

print(credit_scoring.duplicated().sum())

72


In [30]:
# Address the duplicates, if they exist

credit_scoring.drop_duplicates(inplace = True)
credit_scoring.dropna().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
19346,1,4529.316663,43.0,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
19347,0,343937.404131,67.0,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
19348,1,2113.346888,38.0,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
19349,3,3112.481705,38.0,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [31]:
# Last check whether we have any duplicates
print(credit_scoring.duplicated().sum())


0


In [32]:
# Check the size of the dataset that you now have after your first manipulations with it
credit_scoring.shape


(21453, 12)

cheking the loan return rete:


In [33]:
sum_debt=credit_scoring['debt'].sum()
count_debt=credit_scoring['debt'].count()
print("total loan return:", sum_debt)
print("total loan taken: ",count_debt)
print(100*sum_debt/count_debt)

total loan return: 1741
total loan taken:  21453
8.115415093460122


* There is no capital letters, there is no duplicates, 71 row were deleted. 

#### Working with missing values

id provided in education and family: 
* [0, 1, 2, 3, 4] = [bachelor's degree, secondary education, some college, primary education, primary education]
* [0, 1, 2, 3, 4] = [married, civil partnership, widow / widower, divorced, unmarried]

In [34]:
# Find the dictionaries
education_id = credit_scoring['education_id'].unique()
education = credit_scoring['education'].unique()
print(education_id)
for i in education_id:
    print(credit_scoring[credit_scoring['education_id']==i]['education'].unique())

print('**********************')

family_id = credit_scoring['family_status_id'].unique()
print(family_id)
family = credit_scoring['family_status'].unique()
for i in family_id:
    print(credit_scoring[credit_scoring['family_status_id']==i]['family_status'].unique())

[0 1 2 3 4]
["bachelor's degree"]
['secondary education']
['some college']
['primary education']
['graduate degree']
**********************
[0 1 2 3 4]
['married']
['civil partnership']
['widow / widower']
['divorced']
['unmarried']


#### Restoring missing values in `total_income`:

There are missing values at total_income and day_employed columns,I'll fill the missing value by
calculat the mean and the median by diffrent factors and fill the miising values with the outcome. 

In [35]:
# Let's write a function that calculates the age category
def calc_age_category(age):
    if age <= 20:
        return "10's"
    if 20 < age <=30:
        return "20's"
    if 30 < age <=40:
        return "30's"
    if 40 < age <=50:
        return "40's"
    if 50 < age <=60:
        return "50's"
    else:
        return "60's"
    

In [36]:
# Test if the function works
print(calc_age_category(35))
print(calc_age_category(42))
print(calc_age_category(28))
print(calc_age_category(19))

30's
40's
20's
10's


In [37]:
# Creating new column based on function

credit_scoring['age_category'] = credit_scoring['dob_years'].apply(calc_age_category)

In [38]:
# Checking how values in the new column
credit_scoring.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40's
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30's
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30's
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30's
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50's


In [39]:
# Create a table without missing values and print a few of its rows to make sure it looks fine
credit_scoring_notnull = credit_scoring[credit_scoring.notnull()]
credit_scoring_notnull.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40's
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30's
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30's
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30's
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50's
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20's
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40's
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,40's
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30's
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40's


In [40]:
# Look at the mean values for income based on your identified factors
credit_scoring_notnull.groupby('age_category')['total_income'].mean()


age_category
10's    19586.303559
20's    25928.848368
30's    28376.735148
40's    28332.806009
50's    25482.856294
60's    23057.777452
Name: total_income, dtype: float64

In [41]:
# Look at the median values for income based on your identified factors
credit_scoring_notnull.groupby(['age_category','gender']).agg({'total_income':['median','mean']})

total_income              
                          median          mean
age_category gender                           
10's         F        16417.2150  18320.491771
             M        20725.2015  21432.279083
20's         F        20876.6890  23562.735796
             M        26171.1190  29212.363433
30's         F        22345.7290  25745.868532
             M        28342.5610  32529.699158
40's         F        22456.3625  25815.220211
             M        28281.4400  33114.863056
50's         F        20915.2175  24112.012517
             M        25440.1850  29328.941931
60's         F        19373.4450  22493.005456
             M        21021.2510  24909.674913

* I think that income effected from age, gender, end education I used those topicsd to filter and find mean and the median

In [42]:
credit_scoring_notnull.groupby('education').agg({'total_income':['median','mean']})

total_income              
                          median          mean
education                                     
bachelor's degree     28054.5310  33142.802434
graduate degree       25161.5835  27960.024667
primary education     18741.9760  21144.882211
secondary education   21836.5830  24594.503037
some college          25618.4640  29045.443644

In [43]:
credit_scoring_notnull.groupby(['income_type','gender']).agg({'total_income':['median','mean']})

total_income              
                                         median          mean
income_type                 gender                           
business                    F        25737.7920  29476.586824
                            M        31491.0080  37284.334474
civil servant               F        21917.1980  24908.849830
                            M        29754.3915  34036.170503
employee                    F        20898.4980  23818.105441
                            M        25945.7880  28956.563225
entrepreneur                F        79866.1030  79866.103000
                            M               NaN           NaN
paternity / maternity leave F         8612.6610   8612.661000
retiree                     F        18529.2465  21446.167378
                            M        20918.3620  24117.476626
student                     M        15712.2600  15712.260000
unemployed                  F        32435.6020  32435.602000
                            M         9593.1190   9593.119000

* I believe the job and gender define income in the best way, I think the mean reflect the reality better then the median.

In [44]:
#  Write a function that we will use for filling in missing values
def fill_missing(df,column_to_fix, column_to_groupby):
    return df.groupby(column_to_groupby)[column_to_fix].transform(lambda grp: grp.fillna(np.mean(grp)))   
        

In [45]:
# Check if it works
x = fill_missing(credit_scoring,'total_income', ['income_type','gender'])
print(x)
print('**********************')
print(x[x.isnull() == True])
print(len(credit_scoring['total_income']))

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21520    35966.698
21521    24959.969
21522    14347.610
21523    39054.888
21524    13127.587
Name: total_income, Length: 21453, dtype: float64
**********************
5936   NaN
Name: total_income, dtype: float64
21453


In [46]:
# Check if we got any errors
credit_scoring['total_income'] = fill_missing(credit_scoring,'total_income', ['income_type','gender'])

print(credit_scoring[credit_scoring['total_income'].isnull()==True])
print(credit_scoring['total_income'].isnull().sum())

      children  days_employed  dob_years          education  education_id  \
5936         0            NaN       58.0  bachelor's degree             0   

     family_status  family_status_id gender   income_type  debt  total_income  \
5936       married                 0      M  entrepreneur     0           NaN   

                          purpose age_category  
5936  buy residential real estate         50's  
1


* There's only one male entrepreneur on credit scoring with Nan in Total income, the function was anabled to find the mean for its 1 group, I'll  insert the mean value female entrepreneur manualy

In [47]:
credit_scoring.loc[credit_scoring['income_type'] == 'entrepreneur' ,'total_income']=79866.10300
print(credit_scoring[credit_scoring['total_income'].isnull()==True])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, age_category]
Index: []


In [48]:
# Checking the number of entries in the columns
print(len(credit_scoring['total_income']))
print(len(credit_scoring))


21453
21453


####  Restoring values in `days_employed`:

In [49]:
# Distribution of `days_employed` medians based on your identified parameters
credit_scoring_notnull.groupby(['income_type','gender']).agg({'days_employed':['median','mean']})


days_employed               
                                           median           mean
income_type                 gender                              
business                    F         1584.349426    2176.838098
                            M         1472.891912    2001.609022
civil servant               F         2705.835929    3516.054217
                            M         2659.328964    3080.630224
employee                    F         1718.507316    2523.264675
                            M         1362.728215    2018.419861
entrepreneur                F          520.848083     520.848083
                            M                 NaN            NaN
paternity / maternity leave F         3296.759962    3296.759962
retiree                     F       366182.933484  365391.231569
                            M       361781.511144  363295.485985
student                     M          578.751554     578.751554
unemployed                  F       395302.838654  395302.838654
                            M       337524.466835  337524.466835

Im going to use the mean, in most cases it very close to the median, and overall it reflect the realty better

In [50]:
# Check that the function works
x = fill_missing(credit_scoring,'days_employed', ['income_type','gender'])
print(x)
print('**********************')
print(x[x.isnull() == True])
print(len(credit_scoring['days_employed']))

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21453, dtype: float64
**********************
5936   NaN
Name: days_employed, dtype: float64
21453


In [51]:
# Apply function to the income_type
credit_scoring['days_employed'] = fill_missing(credit_scoring,'days_employed', ['income_type','gender'])

In [52]:
# Check if function worked

print(credit_scoring[credit_scoring['days_employed'].isnull()==True])

      children  days_employed  dob_years          education  education_id  \
5936         0            NaN       58.0  bachelor's degree             0   

     family_status  family_status_id gender   income_type  debt  total_income  \
5936       married                 0      M  entrepreneur     0     79866.103   

                          purpose age_category  
5936  buy residential real estate         50's  


In [53]:
# Replacing missing values
credit_scoring.loc[credit_scoring['income_type'] == 'entrepreneur' ,'days_employed']=520.848083
print(credit_scoring[credit_scoring['total_income'].isnull()==True])


Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, age_category]
Index: []


In [54]:
# Check the entries in all columns - make sure we fixed all missing values
print(len(credit_scoring['days_employed']))
print(len(credit_scoring))


21453
21453


#### Categorization of data:

In [55]:
# Print the values for your selected data for categorization

credit_scoring['purpose']


0          purchase of the house
1                   car purchase
2          purchase of the house
3        supplementary education
4              to have a wedding
                  ...           
21520       housing transactions
21521          purchase of a car
21522                   property
21523          buying my own car
21524               to buy a car
Name: purpose, Length: 21453, dtype: object

In [56]:
# Check the unique values

credit_scoring['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

 Main groups identified based on the unique values: real estate, education, hous, car, wedding, property, university

In [57]:
import nltk
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

english_stemmer = SnowballStemmer('english') 

topics = [
    'real estate',
    'education',
    'hous',
    'car',
    'wedding',
    'property',
    'university'
]

stem_topics=[]
for word in topics:
    stemmed_word = english_stemmer.stem(word)
    stem_topics.append(stemmed_word)
    
wordnet_lemma = WordNetLemmatizer()
lemma_topics= [wordnet_lemma.lemmatize(word, pos = 'n') for word in topics]
print(stem_topics)
print(lemma_topics)


['real est', 'educ', 'hous', 'car', 'wed', 'properti', 'univers']
['real estate', 'education', 'hous', 'car', 'wedding', 'property', 'university']


In [58]:
# Let's write a function to categorize the data based on common topics
def  categorize_purpose (purpose):
    stemmed_words=[]
    for word in purpose.split(" "):
        stemmed_words.append(english_stemmer.stem(word)) 
    if 'educ' in stemmed_words or 'univers' in stemmed_words:
        return 'education'
    elif 'hous' in stemmed_words:
        return 'house'
    elif 'car' in stemmed_words:
        return 'car'
    elif 'wed' in stemmed_words:
        return 'wedding'
    elif 'properti' in stemmed_words:
        return 'properti'
    else : 
        return 'real_estate'
    


check if the categorize_purpose function work:

In [59]:
purpose1 = 'buy real estate'
purpose2 = 'to buy a car'
purpose3 = 'housing renovation'
purpose4 = 'going to university'
purpose5 = 'having a wedding'
print(categorize_purpose(purpose1))
print(categorize_purpose(purpose2))
print(categorize_purpose(purpose3))
print(categorize_purpose(purpose4))
print(categorize_purpose(purpose5))

real_estate
car
house
education
wedding


In [60]:
# Create a column with the categories and count the values for them

credit_scoring['categorize_purpose'] = credit_scoring['purpose'].apply(categorize_purpose)
print(credit_scoring['categorize_purpose'].unique())
print(credit_scoring['categorize_purpose'].value_counts()/len(credit_scoring))

['house' 'car' 'education' 'wedding' 'real_estate' 'properti']
real_estate    0.208083
car            0.200718
education      0.187060
house          0.177551
properti       0.118305
wedding        0.108283
Name: categorize_purpose, dtype: float64


In [61]:
# Looking through all the numerical data in your selected column for categorization
print('max total income', credit_scoring['total_income'].max())
print('min total income',credit_scoring['total_income'].min())
print('mean total income',credit_scoring['total_income'].mean())
print('median total income',credit_scoring['total_income'].median())


max total income 362496.645
min total income 3306.762
mean total income 26780.946004768422
median total income 23818.1054405171


In [62]:
# Creating function for categorizing into different numerical groups based on ranges
def total_income_categorize (total_income):
    if total_income >= 26781:
        return 'above average'
    elif 23818 <= total_income < 26781:
        return 'between median to average'
    else:
        return  'below median'


In [63]:
# Creating column with categories
credit_scoring['total_income_categorize'] = credit_scoring['total_income'].apply(total_income_categorize)
print(credit_scoring['total_income_categorize'].unique())

['above average' 'below median' 'between median to average']


In [64]:
# Count each categories values to see the distribution
print(credit_scoring['total_income_categorize'].unique())
print(credit_scoring['total_income_categorize'].value_counts()/len(credit_scoring))

['above average' 'below median' 'between median to average']
below median                 0.483615
above average                0.391647
between median to average    0.124738
Name: total_income_categorize, dtype: float64


#### Checking the Hypotheses:

**Is there a correlation between having children and paying back on time?**

In [65]:
# Check the children data and paying back on time
children_debt=credit_scoring.pivot_table(index='children',values ='debt' ,aggfunc =['sum','count'])
print(children_debt)
# Calculating default-rate based on the number of children
children_debt['default_rate'] = children_debt['sum']/children_debt['count']
print(children_debt)
print(children_debt.sum())

           sum  count
          debt   debt
children             
0         1063  14090
1          445   4855
2          202   2128
3           27    330
4            4     41
5            0      9
           sum  count default_rate
          debt   debt             
children                          
0         1063  14090     0.075444
1          445   4855     0.091658
2          202   2128     0.094925
3           27    330     0.081818
4            4     41     0.097561
5            0      9     0.000000
sum           debt     1741.000000
count         debt    21453.000000
default_rate              0.441406
dtype: float64


**Conclusion**
people with 4 kids pay back their loans in 9.8% rate 
people with 2 kids pay back their loans in 9.5% rate 
people with 1 kids pay back their loans in 9.2% rate 
people with 3 kids pay back their loans in 8.2% rate 
people with 5 kids dont pay back their loans, but ther's only 9 people with 5 kids that took a loan 


**Is there a correlation between family status and paying back on time?**

In [66]:
# Check the family status data and paying back on time
family_status_debt=credit_scoring.pivot_table(index='family_status',values ='debt' ,aggfunc =['sum','count'])
print(family_status_debt)
# Calculating default-rate based on family status
family_status_debt['default_rate'] = family_status_debt['sum']/family_status_debt['count']
print(family_status_debt)

print(family_status_debt.sum())

                   sum  count
                  debt   debt
family_status                
civil partnership  388   4150
divorced            85   1195
married            931  12339
unmarried          274   2810
widow / widower     63    959
                   sum  count default_rate
                  debt   debt             
family_status                             
civil partnership  388   4150     0.093494
divorced            85   1195     0.071130
married            931  12339     0.075452
unmarried          274   2810     0.097509
widow / widower     63    959     0.065693
sum           debt     1741.000000
count         debt    21453.000000
default_rate              0.403278
dtype: float64


**Conclusion**
people in civil parthership return thier loans in 9.3% rate
unmerried people return thier loans in 9.7 % rate
married return thier loans in 7.5 % rate
divorced return thier loans in 7.1 % rate
widow / widowe return thier loans in 6.5 % rate

**Is there a correlation between income level and paying back on time?**

In [67]:
# Check the income level data and paying back on time
income_level_debt=credit_scoring.pivot_table(index='total_income_categorize',values ='debt' ,aggfunc =['sum','count'])
print(income_level_debt)
# Calculating default-rate based on income level
income_level_debt['default_rate'] = income_level_debt['sum']/income_level_debt['count']
print(income_level_debt)
print(income_level_debt.sum())


                           sum  count
                          debt   debt
total_income_categorize              
above average              648   8402
below median               863  10375
between median to average  230   2676
                           sum  count default_rate
                          debt   debt             
total_income_categorize                           
above average              648   8402     0.077124
below median               863  10375     0.083181
between median to average  230   2676     0.085949
sum           debt     1741.000000
count         debt    21453.000000
default_rate              0.246254
dtype: float64


**Conclusion**
people in between median and average of income are tend to return their loan -  8.6% rate
people who earn below medianof income are tend to return their loan in 8.3% rate  
people who earn above the average income are tend to return their loan only in 7.7% rate- the least
the distribution I got close to the genral returning loan distribution(8%), I dont think I can conclud somthing from that.

**How does credit purpose affect the default rate?**

In [68]:
# Check the percentages for default rate for each credit purpose and analyze them

credit_purpose_debt=credit_scoring.pivot_table(index='categorize_purpose',values ='debt' ,aggfunc =['sum','count'])
print(credit_purpose_debt)
# Calculating default-rate based on income level
credit_purpose_debt['default_rate'] =credit_purpose_debt['sum']/credit_purpose_debt['count']
print(credit_purpose_debt)
print(credit_purpose_debt.sum() )

                    sum count
                   debt  debt
categorize_purpose           
car                 403  4306
education           370  4013
house               256  3809
properti            190  2538
real_estate         336  4464
wedding             186  2323
                    sum count default_rate
                   debt  debt             
categorize_purpose                        
car                 403  4306     0.093590
education           370  4013     0.092200
house               256  3809     0.067209
properti            190  2538     0.074862
real_estate         336  4464     0.075269
wedding             186  2323     0.080069
sum           debt     1741.0000
count         debt    21453.0000
default_rate              0.4832
dtype: float64


**Conclusion**
people who took loan for car return thier loan in 9.4% rate
people who took loan for education return their loan in 9.2% rate
people who took loan for wedding return thier loan in 8% rate
people who took loan for real_estate return thier loan in 7.52% rate
people who took loan for properti  return thier loan in 7.5% rate
people who took loan for house return thier loan in 6.7% rate

again the distribution close to the genral distribution, but I can say that more people who took loan for cars, and education pay back their loans then people who took loan for housing and real estate.



#### General Conclusion 
I got data set with infornation about people who took loans, I filles up the missing vlues correct mis spelling , drop duplicates, and catagorize the data by family status, education,  reason for loan, and employment, and having children. thee is no distinct conclusion, but we see groups who tent to pay their loans more then others:

people with 4 kids pay back their loans in 9.8% rate 
people with 2 kids pay back their loans in 9.5% rate 
people with 1 kids pay back their loans in 9.2% rate 
people who took loan for car return thier loan in 9.4% rate
people who took loan for education return their loan in 9.2% rate
people in civil parthership return thier loans in 9.3% rate
unmerried people return thier loans in 9.7 % rate
people in between median and average of income are tend to return their loan -  8.6% rate